In [2]:
import pandas as pd
import re
from typing import Dict, Union, List

In [3]:
colunas = ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']

# LINHA DE SELECAO DO INPUT
df = pd.read_csv(
    "dataset_clinica20252.csv",
    sep="|",               
    encoding="utf-8"     
)

print(df.shape)
print(df.head(20))

(19800, 6)
      cd_causa             cd_atendimento                 ds_Acao_Judicial  \
0   CIB0500064  0825789-84.2025.8.18.0140  90 - ACAO DE REPARACAO DE DANOS   
1   CIB0505587  1004697-72.2025.8.26.0066  90 - ACAO DE REPARACAO DE DANOS   
2   CIB0508201  0800423-07.2025.8.15.0761  90 - ACAO DE REPARACAO DE DANOS   
3   CIB0514647  1004875-69.2025.8.26.0438  90 - ACAO DE REPARACAO DE DANOS   
4   CIB0500604  0010630-50.2025.8.27.2706  90 - ACAO DE REPARACAO DE DANOS   
5   CIB0511068  0092601-36.2025.8.05.0001  90 - ACAO DE REPARACAO DE DANOS   
6   CIB0515030  0801575-51.2025.8.18.0068  90 - ACAO DE REPARACAO DE DANOS   
7   CIB0515018  0801452-38.2025.8.18.0073  90 - ACAO DE REPARACAO DE DANOS   
8   CIB0515702  0800895-81.2025.8.10.0038  90 - ACAO DE REPARACAO DE DANOS   
9   CIB0514229  0148716-17.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
10  CIB0514230  0148734-38.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
11  CIB0524760  0161843-22.2025.8.04.1000  90 - ACAO 

In [9]:
def extrair_cnpj(texto):
    padroes = [
        # Padrão 1: Formato completo (aceita . ou , como separador, / ou \ como barra)
        # Ex: 60.746.948/0001-12 ou 60,746,948\0001-12
        r'(\d{2}[.,]\d{3}[.,]\d{3}[/\\]\d{4}[-\s]?\d{2})',
        
        # Padrão 2: Formato sem barra (com pontos ou vírgulas)
        # Ex: 60.746.948.0001-12 ou 60,746,948,0001-12
        r'(\d{2}[.,]\d{3}[.,]\d{3}[.,]\d{4}[-\s]?\d{2})',
        
        # Padrão 3: Com espaços ao redor dos separadores
        # Ex: 60.746.948 / 0001-12 ou 60,746,948 \ 0001 - 12
        r'(\d{2}\s*[.,]\s*\d{3}\s*[.,]\s*\d{3}\s*[/\\]\s*\d{4}\s*[-\s]?\s*\d{2})',
        
        # Padrão 4: Com espaços e ponto no lugar de barra
        # Ex: 60.746.948 . 0001-12
        r'(\d{2}\s*[.,]\s*\d{3}\s*[.,]\s*\d{3}\s*[.,]\s*\d{4}\s*[-\s]?\s*\d{2})',
        
        # Padrão 5: Formato flexível (qualquer combinação de separadores)
        # Ex: 60 746 948/0001-12 ou 60.746,948\0001 12
        r'(\d{2}[\s.,]*\d{3}[\s.,]*\d{3}[\s.,/\\]*\d{4}[\s\-]*\d{2})',
        
        # Padrão 6: CNPJ com "nº" ou "n°" ou "no" antes
        # Ex: CNPJ nº 60.746.948/0001-12
        r'(?:CNPJ|cnpj)[\s:]*(?:n[ºo°]?\.?[\s:]*)(\d{2}[.,\s]*\d{3}[.,\s]*\d{3}[/\\.,\s]*\d{4}[-\s]*\d{2})',
        
        # Padrão 7: Apenas 14 dígitos seguidos
        r'\b(\d{14})\b',
    ]
    
    for padrao in padroes:
        match = re.search(padrao, texto)
        if match:
            cnpj_bruto = match.group(1)
            cnpj_limpo = re.sub(r'\D', '', cnpj_bruto)
            
            if len(cnpj_limpo) == 14:
                return cnpj_limpo
    return None 
    
    

In [18]:
def extrair_valor_causa(texto: str) -> Dict[str, Union[str, float, List[str], bool, None]]:
    """
    Extrai o valor da causa de textos de petições judiciais brasileiras.
    
    Versão 3.0 - Melhorada para capturar valores sem formatação completa.
    """
    if not texto or not isinstance(texto, str) or texto.strip() == '':
        return {
            'valor_encontrado': None,
            'valor_numerico': None,
            'padrao_usado': None,
            'confianca': None,
            'todos_valores': [],
            'sucesso': False,
            'qtd_valores': 0
        }

    # Padrões em ordem de prioridade
    padroes = [
        # NOVOS PADRÕES v3.0 (alta confiança) ================================
        
        # Padrão 1: "Dá-se à causa[,] o valor [de] R$" (aceita vírgula após "causa")
        {
            'nome': 'dá-se à causa (com vírgula)',
            'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão|dando-se)\s+(?:à|a)\s+(?:presente\s+)?(?:causa|demanda|ação)\s*,?\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 2: "valor total da pretensão condenatória" / "ou seja, a quantia de"
        {
            'nome': 'valor da pretensão/quantia',
            'regex': r'(?:valor\s+(?:do\s+)?total\s+da\s+pretensão\s+condenatória|ou\s+seja,?\s+a\s+quantia)\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 3: "todos os efeitos de direito" (comum no final)
        {
            'nome': 'para efeitos de direito',
            'regex': r'para\s+(?:todos\s+os\s+)?efeitos\s+de\s+direito[\.,]?\s*R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 4: "Dá-se/dando-se à causa/demanda"
        {
            'nome': 'dá-se/dando-se à causa/demanda',
            'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão|dando-se)\s+(?:à|a)\s+(?:presente\s+)?(?:causa|demanda|ação)\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },

        # Padrão 5: "Valor da causa com por extenso"
        {
            'nome': 'valor da causa com por extenso',
            'regex': r'(?:valor|valor\s+da\s+causa|valor\s+da\s+demanda|valor\s+da\s+ação)\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})\s*\([A-Za-zÀ-ÿ\s]+\)',
            'confianca': 'alta'
        },

        # Padrão 6: "Fins fiscais/alçada antes"
        {
            'nome': 'fins fiscais/alçada antes',
            'regex': r'para\s+(?:os\s+)?fins\s+(?:fiscais|de\s+alçada|fiscais\s+e\s+de\s+alçada)[\.,]?\s*R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 7: "Dando-se à causa"
        {
            'nome': 'dando-se à causa',
            'regex': r'dando-se\s+(?:à|a)\s+causa\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 8: "Dar-se-á à causa"
        {
            'nome': 'dar-se-á à causa',
            'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão)\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 9: "Valor da causa explícito"
        {
            'nome': 'valor da causa explícito',
            'regex': r'valor\s+da\s+causa[:\s]+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 10: "Atribui-se à causa"
        {
            'nome': 'atribui-se à causa',
            'regex': r'atribui[r-]?se\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # Padrão 11: "Seção valor da causa"
        {
            'nome': 'seção valor da causa',
            'regex': r'(?:DO|DA)\s+VALOR\s+DA\s+CAUSA.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        
        # PADRÕES DE MÉDIA CONFIANÇA ==========================================
        
        {
            'nome': 'contexto causa-valor',
            'regex': r'(?:atribui|atribuir|atribuído|atribuindo).*?causa.*?valor.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'média'
        },
        
        {
            'nome': 'fins fiscais (contexto)',
            'regex': r'(?:para\s+fins|para\s+efeitos).*?(?:fiscais|meramente\s+fiscais).*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'média'
        },
    ]

    resultado = {
        'valor_encontrado': None,
        'valor_numerico': None,
        'padrao_usado': None,
        'confianca': None,
        'todos_valores': [],
        'sucesso': False,
        'qtd_valores': 0
    }

    padrao_geral = r'R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})'
    todos_valores_brutos = re.findall(padrao_geral, texto)
    
    todos_valores = []
    for valor in todos_valores_brutos:
        # Se não tem ponto, adicionar formatação
        if '.' not in valor:
            # Extrair apenas números
            numeros = re.sub(r'\D', '', valor)
            if len(numeros) >= 3:  # Tem pelo menos 3 dígitos
                parte_inteira = numeros[:-2]
                centavos = numeros[-2:]
                
                if len(parte_inteira) > 3:
                    parte_inteira_formatada = ''
                    for i, digito in enumerate(reversed(parte_inteira)):
                        if i > 0 and i % 3 == 0:
                            parte_inteira_formatada = '.' + parte_inteira_formatada
                        parte_inteira_formatada = digito + parte_inteira_formatada
                    valor_formatado = f"{parte_inteira_formatada},{centavos}"
                else:
                    valor_formatado = f"{parte_inteira},{centavos}"
                
                todos_valores.append(valor_formatado)
            else:
                todos_valores.append(valor)
        else:
            todos_valores.append(valor)
    
    resultado['todos_valores'] = todos_valores
    resultado['qtd_valores'] = len(todos_valores)

    if not todos_valores:
        return resultado

    for padrao in padroes:
        regex_modificado = padrao['regex'].replace(
            r'([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            r'([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})'
        )
        
        match = re.search(regex_modificado, texto, re.IGNORECASE | re.DOTALL | re.MULTILINE)
        if match:
            valor_str = match.group(1)
            
            if '.' not in valor_str:
                numeros = re.sub(r'\D', '', valor_str)
                if len(numeros) >= 3:
                    parte_inteira = numeros[:-2]
                    centavos = numeros[-2:]
                    
                    if len(parte_inteira) > 3:
                        parte_inteira_formatada = ''
                        for i, digito in enumerate(reversed(parte_inteira)):
                            if i > 0 and i % 3 == 0:
                                parte_inteira_formatada = '.' + parte_inteira_formatada
                            parte_inteira_formatada = digito + parte_inteira_formatada
                        valor_str = f"{parte_inteira_formatada},{centavos}"
                    else:
                        valor_str = f"{parte_inteira},{centavos}"
            
            resultado['valor_encontrado'] = f"R$ {valor_str}"
            resultado['padrao_usado'] = padrao['nome']
            resultado['confianca'] = padrao['confianca']
            resultado['sucesso'] = True
            
            try:
                resultado['valor_numerico'] = float(valor_str.replace('.', '').replace(',', '.'))
            except ValueError:
                resultado['valor_numerico'] = None
            
            return resultado

    # Heurística: último valor encontrado (baixa confiança)
    valor_str = todos_valores[-1]
    resultado['valor_encontrado'] = f"R$ {valor_str}"
    resultado['padrao_usado'] = 'último valor (heurística)'
    resultado['confianca'] = 'baixa'
    resultado['sucesso'] = True
    
    try:
        resultado['valor_numerico'] = float(valor_str.replace('.', '').replace(',', '.'))
    except ValueError:
        resultado['valor_numerico'] = None

    return resultado

In [19]:
df['valor_encontrado'] = df['ds_Pedidos'].apply(lambda x: extrair_valor_causa(str(x))['valor_encontrado'])
df['valor_numerico'] = df['ds_Pedidos'].apply(lambda x: extrair_valor_causa(str(x))['valor_numerico'])
df['padrao_usado'] = df['ds_Pedidos'].apply(lambda x: extrair_valor_causa(str(x))['padrao_usado'])
df['confianca'] = df['ds_Pedidos'].apply(lambda x: extrair_valor_causa(str(x))['confianca'])
df['todos_valores'] = df['ds_Pedidos'].apply(lambda x: extrair_valor_causa(str(x))['todos_valores'])
df['sucesso'] = df['ds_Pedidos'].apply(lambda x: extrair_valor_causa(str(x))['sucesso'])
df['qtd_valores'] = df['ds_Pedidos'].apply(lambda x: extrair_valor_causa(str(x))['qtd_valores'])

print("✅ Processamento concluído!")

# Ver os primeiros resultados
print("\n📊 Primeiros 10 resultados:")
print(df[['valor_encontrado', 'valor_numerico', 'confianca', 'padrao_usado']].head(10))

# Estatísticas
print("\n📈 Estatísticas:")
print(f"Total de processos: {len(df)}")
print(f"Extrações bem-sucedidas: {df['sucesso'].sum()}")
print(f"Taxa de sucesso: {(df['sucesso'].sum() / len(df)) * 100:.1f}%")
print(f"\nDistribuição de confiança:")
print(df['confianca'].value_counts())

✅ Processamento concluído!

📊 Primeiros 10 resultados:
  valor_encontrado  valor_numerico confianca                    padrao_usado
0     R$ 21.906,64        21906.64      alta     dá-se à causa (com vírgula)
1     R$ 10.000,00        10000.00      alta     dá-se à causa (com vírgula)
2     R$ 11.409,08        11409.08      alta     dá-se à causa (com vírgula)
3      R$ 1.256,04         1256.04      alta  valor da causa com por extenso
4     R$ 15.215,42        15215.42      alta     dá-se à causa (com vírgula)
5     R$ 15.000,00        15000.00      alta     dá-se à causa (com vírgula)
6      R$ 5.159,84         5159.84      alta     dá-se à causa (com vírgula)
7             None             NaN      None                            None
8             None             NaN      None                            None
9     R$ 16.804,00        16804.00      alta     dá-se à causa (com vírgula)

📈 Estatísticas:
Total de processos: 19800
Extrações bem-sucedidas: 19467
Taxa de sucesso: 98.3%



In [11]:
df['cnpj_extraido'] = df['ds_Qualificacao'].apply(lambda x: extrair_cnpj(x))

print(f"Total de CNPJs não encontrados: {len(df[df['cnpj_extraido'].isnull()])}")
print(f"Quantidade de CNPJs extraídos com sucesso: {len(df) - len(df[df['cnpj_extraido'].isnull()])}")
print(f"Porcentagem de CNPJs extraídos com sucesso: {((len(df) - len(df[df['cnpj_extraido'].isnull()])) / len(df)) * 100:.1f}%")

Total de CNPJs não encontrados: 606
Quantidade de CNPJs extraídos com sucesso: 19194
Porcentagem de CNPJs extraídos com sucesso: 96.9%


In [17]:
pd.set_option('display.max_colwidth', None)

filtro_baixa = df['confianca'].fillna('').str.lower().eq('baixa')
vis = df.loc[filtro_baixa, ['cd_atendimento', 'ds_Pedidos']].reset_index(drop=True).head(20)

display(vis) 

cd_atendimento  \
0   0800248-16.2025.8.18.0054   
1   0801427-95.2025.8.20.5108   
2   5000974-63.2025.8.13.0607   
3   0800468-62.2025.8.10.0207   
4   0801411-08.2025.8.20.5120   
5   0802713-11.2025.8.20.5108   
6   5002590-61.2025.8.13.0611   
7   0800366-73.2025.8.18.0027   
8   0177511-33.2025.8.04.1000   
9   0800629-21.2025.8.18.0055   
10  0800378-90.2025.8.14.0062   
11  0801864-17.2025.8.15.0181   
12  0013467-12.2025.8.16.0014   
13  0802069-46.2025.8.15.0181   
14  0817133-41.2025.8.18.0140   
15  0800014-26.2025.8.15.0601   
16  0801339-16.2025.8.14.0067   
17  0800458-85.2025.8.18.0048   
18  0802517-82.2025.8.18.0036   
19  3002028-06.2025.8.06.0151   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [20]:
def analisa_regex(pedido: str, qualificacao: str):
    """
    Analisa texto de pedido e qualificação para extrair valor da causa e CNPJ.
    
    Args:
        pedido (str): Texto da coluna ds_Pedidos
        qualificacao (str): Texto da coluna ds_Qualificacao
        
    Returns:
        dict: {
            'valor_numerico': float ou None,
            'confianca': 'alta', 'média', 'baixa' ou None,
            'cnpj': str (14 dígitos) ou None
        }
    """
    
    # ========================================================================
    # EXTRAÇÃO DO VALOR DA CAUSA
    # ========================================================================
    
    valor_numerico = None
    confianca = None
    
    if pedido and isinstance(pedido, str) and pedido.strip() != '':
        padroes = [
            # NOVOS PADRÕES v3.0 (alta confiança) ================================
            
            # Padrão 1: "Dá-se à causa[,] o valor [de] R$" (aceita vírgula após "causa")
            {
                'nome': 'dá-se à causa (com vírgula)',
                'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão|dando-se)\s+(?:à|a)\s+(?:presente\s+)?(?:causa|demanda|ação)\s*,?\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 2: "valor total da pretensão condenatória" / "ou seja, a quantia de"
            {
                'nome': 'valor da pretensão/quantia',
                'regex': r'(?:valor\s+(?:do\s+)?total\s+da\s+pretensão\s+condenatória|ou\s+seja,?\s+a\s+quantia)\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 3: "todos os efeitos de direito" (comum no final)
            {
                'nome': 'para efeitos de direito',
                'regex': r'para\s+(?:todos\s+os\s+)?efeitos\s+de\s+direito[\.,]?\s*R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 4: "Dá-se/dando-se à causa/demanda"
            {
                'nome': 'dá-se/dando-se à causa/demanda',
                'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão|dando-se)\s+(?:à|a)\s+(?:presente\s+)?(?:causa|demanda|ação)\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 5: "Valor da causa com por extenso"
            {
                'nome': 'valor da causa com por extenso',
                'regex': r'(?:valor|valor\s+da\s+causa|valor\s+da\s+demanda|valor\s+da\s+ação)\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})\s*\([A-Za-zÀ-ÿ\s]+\)',
                'confianca': 'alta'
            },
            
            # Padrão 6: "Fins fiscais/alçada antes"
            {
                'nome': 'fins fiscais/alçada antes',
                'regex': r'para\s+(?:os\s+)?fins\s+(?:fiscais|de\s+alçada|fiscais\s+e\s+de\s+alçada)[\.,]?\s*R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 7: "Dando-se à causa"
            {
                'nome': 'dando-se à causa',
                'regex': r'dando-se\s+(?:à|a)\s+causa\s+o\s+valor\s+(?:de\s+)?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 8: "Dar-se-á à causa"
            {
                'nome': 'dar-se-á à causa',
                'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão)\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 9: "Valor da causa explícito"
            {
                'nome': 'valor da causa explícito',
                'regex': r'valor\s+da\s+causa[:\s]+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 10: "Atribui-se à causa"
            {
                'nome': 'atribui-se à causa',
                'regex': r'atribui[r-]?se\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # Padrão 11: "Seção valor da causa"
            {
                'nome': 'seção valor da causa',
                'regex': r'(?:DO|DA)\s+VALOR\s+DA\s+CAUSA.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'alta'
            },
            
            # PADRÕES DE MÉDIA CONFIANÇA ==========================================
            
            {
                'nome': 'contexto causa-valor',
                'regex': r'(?:atribui|atribuir|atribuído|atribuindo).*?causa.*?valor.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'média'
            },
            
            {
                'nome': 'fins fiscais (contexto)',
                'regex': r'(?:para\s+fins|para\s+efeitos).*?(?:fiscais|meramente\s+fiscais).*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                'confianca': 'média'
            },
        ]
        
        padrao_geral = r'R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})'
        todos_valores_brutos = re.findall(padrao_geral, pedido)
        
        todos_valores = []
        for valor in todos_valores_brutos:
            if '.' not in valor:
                numeros = re.sub(r'\D', '', valor)
                if len(numeros) >= 3:
                    parte_inteira = numeros[:-2]
                    centavos = numeros[-2:]
                    
                    if len(parte_inteira) > 3:
                        parte_inteira_formatada = ''
                        for i, digito in enumerate(reversed(parte_inteira)):
                            if i > 0 and i % 3 == 0:
                                parte_inteira_formatada = '.' + parte_inteira_formatada
                            parte_inteira_formatada = digito + parte_inteira_formatada
                        valor_formatado = f"{parte_inteira_formatada},{centavos}"
                    else:
                        valor_formatado = f"{parte_inteira},{centavos}"
                    
                    todos_valores.append(valor_formatado)
                else:
                    todos_valores.append(valor)
            else:
                todos_valores.append(valor)
        
        if todos_valores:
            for padrao in padroes:
                regex_modificado = padrao['regex'].replace(
                    r'([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
                    r'([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})'
                )
                
                match = re.search(regex_modificado, pedido, re.IGNORECASE | re.DOTALL | re.MULTILINE)
                if match:
                    valor_str = match.group(1)
                    
                    if '.' not in valor_str:
                        numeros = re.sub(r'\D', '', valor_str)
                        if len(numeros) >= 3:
                            parte_inteira = numeros[:-2]
                            centavos = numeros[-2:]
                            
                            if len(parte_inteira) > 3:
                                parte_inteira_formatada = ''
                                for i, digito in enumerate(reversed(parte_inteira)):
                                    if i > 0 and i % 3 == 0:
                                        parte_inteira_formatada = '.' + parte_inteira_formatada
                                    parte_inteira_formatada = digito + parte_inteira_formatada
                                valor_str = f"{parte_inteira_formatada},{centavos}"
                            else:
                                valor_str = f"{parte_inteira},{centavos}"
                    
                    confianca = padrao['confianca']
                    
                    try:
                        valor_numerico = float(valor_str.replace('.', '').replace(',', '.'))
                    except ValueError:
                        valor_numerico = None
                    
                    break
            
            if valor_numerico is None and todos_valores:
                valor_str = todos_valores[-1]
                confianca = 'baixa'
                
                try:
                    valor_numerico = float(valor_str.replace('.', '').replace(',', '.'))
                except ValueError:
                    valor_numerico = None
    
    # ========================================================================
    # EXTRAÇÃO DO CNPJ
    # ========================================================================
    
    cnpj = None
    
    if qualificacao and isinstance(qualificacao, str):
        padroes_cnpj = [
            # Padrão 1: Formato completo (aceita . ou , como separador, / ou \ como barra)
            # Ex: 60.746.948/0001-12 ou 60,746,948\0001-12
            r'(\d{2}[.,]\d{3}[.,]\d{3}[/\\]\d{4}[-\s]?\d{2})',
            
            # Padrão 2: Formato sem barra (com pontos ou vírgulas)
            # Ex: 60.746.948.0001-12 ou 60,746,948,0001-12
            r'(\d{2}[.,]\d{3}[.,]\d{3}[.,]\d{4}[-\s]?\d{2})',
            
            # Padrão 3: Com espaços ao redor dos separadores
            # Ex: 60.746.948 / 0001-12 ou 60,746,948 \ 0001 - 12
            r'(\d{2}\s*[.,]\s*\d{3}\s*[.,]\s*\d{3}\s*[/\\]\s*\d{4}\s*[-\s]?\s*\d{2})',
            
            # Padrão 4: Com espaços e ponto no lugar de barra
            # Ex: 60.746.948 . 0001-12
            r'(\d{2}\s*[.,]\s*\d{3}\s*[.,]\s*\d{3}\s*[.,]\s*\d{4}\s*[-\s]?\s*\d{2})',
            
            # Padrão 5: Formato flexível (qualquer combinação de separadores)
            # Ex: 60 746 948/0001-12 ou 60.746,948\0001 12
            r'(\d{2}[\s.,]*\d{3}[\s.,]*\d{3}[\s.,/\\]*\d{4}[\s\-]*\d{2})',
            
            # Padrão 6: CNPJ com "nº" ou "n°" ou "no" antes
            # Ex: CNPJ nº 60.746.948/0001-12
            r'(?:CNPJ|cnpj)[\s:]*(?:n[ºo°]?\.?[\s:]*)(\d{2}[.,\s]*\d{3}[.,\s]*\d{3}[/\\.,\s]*\d{4}[-\s]*\d{2})',
            
            # Padrão 7: Apenas 14 dígitos seguidos
            r'\b(\d{14})\b',
        ]
        
        for padrao_cnpj in padroes_cnpj:
            match = re.search(padrao_cnpj, qualificacao)
            if match:
                cnpj_bruto = match.group(1)
                cnpj_limpo = re.sub(r'\D', '', cnpj_bruto)
                
                if len(cnpj_limpo) == 14:
                    cnpj = cnpj_limpo
                    break
    
    return {
        'valor_numerico': valor_numerico,
        'confianca': confianca,
        'cnpj': cnpj
    }
